### Librairies

In [1]:
import os
import pandas as pd
import requests

### Authentification pour récupérer le token

In [2]:
headers_auth = {
    "Content-Type": "application/x-www-form-urlencoded"
}

data_auth = {
    "grant_type": "client_credentials",
    "client_id": os.getenv("SPOTIFY_CLIENT_ID"),
    "client_secret": os.getenv("SPOTIFY_CLIENT_SECRET")
}

url_auth = "https://accounts.spotify.com/api/token"

try:
    response_auth = requests.post(url_auth, headers=headers_auth, data=data_auth)
    response_auth.raise_for_status()  # lève une exception si statut != 200

    # Extraction du token
    access_token = response_auth.json().get("access_token")

except requests.exceptions.RequestException as error:
    print("Exception during authentication:", error)
    access_token = None

In [10]:
headers_auth = {"Authorization": f"Bearer {access_token}"}

artist_id = "7jy3rLJdDQY21OgRLCZ9sD"  # Foo Fighters

### Appel API artist

In [11]:
if access_token:
    print("Get artist informations ... ")

    url_artist = f"https://api.spotify.com/v1/artists/{artist_id}"

    try:
        response_artist = requests.get(url_artist, headers=headers_auth)
        response_artist.raise_for_status()

        artist_json = response_artist.json()

        artist_data = {
            "id": artist_json["id"],
            "name": artist_json["name"],
            "url": artist_json["external_urls"]["spotify"],
            "image_url": artist_json["images"][0]["url"],
            "popularity": artist_json["popularity"],
            "genres": artist_json["genres"],  # laisse en liste
            "total_followers": artist_json["followers"]["total"],
            "uri": artist_json["uri"]
        }

        # Création du DataFrame
        df_artist = pd.DataFrame([artist_data])

        # Sauvegarde CSV
        df_artist.to_csv("../datas/artist.csv", index=False)

        print("... OK")

    except requests.exceptions.RequestException as error:
        print("Exception during artist data retrieval:", error)

Get artist informations ... 
... OK


### Appel API Albums

In [18]:
if access_token:
    print("Get albums informations ... ")

    params_albums = {
        "include_groups": "album",
        "limit": 50,
        "offset": 0
    }

    url_albums = f"https://api.spotify.com/v1/artists/{artist_id}/albums"

    try:
        response_albums = requests.get(url_albums, headers=headers_auth, params=params_albums)
        response_albums.raise_for_status()

        albums_json = response_albums.json()

        if albums_json['next'] is None:
            print("Total number of albums :", albums_json['total'])
            
            # On initialise une liste pour stocker les lignes du DataFrame
            albums_data = []

            for album in albums_json["items"]:
                data = {
                    "name": album["name"],
                    "release_date": album["release_date"],
                    "release_date_precision": album["release_date_precision"],
                    "total_tracks": album["total_tracks"],
                    "url": album["external_urls"]["spotify"],
                    "image_url": album["images"][0]["url"],
                    "available_markets": album["available_markets"],
                    "uri": album["uri"]
                }
                albums_data.append(data)

            # Création du DataFrame
            df_albums = pd.DataFrame(albums_data)

            # Sauvegarde CSV
            df_albums.to_csv("../datas/albums.csv", index=False)

        print("... OK")

    except requests.exceptions.RequestException as error:
        print("Exception during albums data retrieval:", error)

Get albums informations ... 
Total number of albums : 14
... OK
